# First attempt

In [5]:
import pandas as pd

[Swear words list](http://www.slate.com/blogs/lexicon_valley/2013/09/11/top_swear_words_most_popular_curse_words_on_facebook.html)

In [6]:
SWEAR_WORDS = 'shit, fuck, damn, bitch, crap, piss, dick, darn, cock, pussy, asshole, fag, bastard, douche'.split(', ')

## Read Data

[Dataset source](https://www.kaggle.com/kazanova/sentiment140/version/2)

In [69]:
%%time 
tweets = pd.read_csv('1mil-tweets.csv', encoding='latin-1',
                     header=None, names=['polarity', 'post_id', 'date', 'flag', 'user_id', 'content']).drop(['polarity', 'flag'], axis=1)

CPU times: user 3.49 s, sys: 329 ms, total: 3.82 s
Wall time: 3.82 s


In [70]:
tweets.head()

,post_id,date,user_id,content
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


In [71]:
tweets_dev = tweets.head(10_000)

## Pre-processing

In [72]:
from textblob import TextBlob
from dateutil.parser import parse as parse_date

In [73]:
import warnings
warnings.simplefilter('ignore')

### Content-related

In [74]:
tweets_dev['content_len'] = tweets_dev.content.str.len()

In [75]:
%time tweets_dev['content_polarity'] = tweets_dev.content.apply(lambda t: TextBlob(t).polarity)

CPU times: user 2.21 s, sys: 5.72 ms, total: 2.21 s
Wall time: 2.22 s


In [76]:
%time tweets_dev['content_subjectivity'] = tweets_dev.content.apply(lambda t: TextBlob(t).subjectivity)

CPU times: user 2.12 s, sys: 4.76 ms, total: 2.12 s
Wall time: 2.13 s


### Date-related

In [77]:
# parse dates
%time tweets_dev.date = tweets_dev.date.apply(parse_date)

CPU times: user 1.3 s, sys: 10.5 ms, total: 1.31 s
Wall time: 1.31 s


In [79]:
%time tweets_dev['day_of_week'] = tweets_dev.date.apply(lambda d: int(d.strftime('%w')))

CPU times: user 1.75 s, sys: 4.93 ms, total: 1.76 s
Wall time: 1.75 s


In [80]:
%time tweets_dev['hour_of_day'] = tweets_dev.date.apply(lambda d: int(d.strftime('%H')))

CPU times: user 1.66 s, sys: 3.65 ms, total: 1.67 s
Wall time: 1.66 s


### Swear-word related

In [82]:
def count_swears(text: str) -> int:
    """
    >>> count_swears('damn damn abc crap 123')
    3
    """
    words = text.split()  # most basic
    return sum(words.count(swear) for swear in SWEAR_WORDS)

In [84]:
tweets_dev['n_swears'] = tweets_dev.content.apply(count_swears)

In [85]:
tweets_dev['swear_presence'] = tweets_dev.n_swears > 0

In [89]:
f'{tweets_dev.swear_presence.mean():.2%} of the tweets contain at least one swear word'

'1.36% of the tweets contain at least one swear word'

In [90]:
tweets_dev.head()

,post_id,date,user_id,content,content_len,content_polarity,content_subjectivity,day_of_week,hour_of_day,n_swears,swear_presence
0,1467810369,2009-04-06 22:19:45-07:00,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",115,0.216667,0.633333,1,22,0,False
1,1467810672,2009-04-06 22:19:49-07:00,scotthamilton,is upset that he can't update his Facebook by ...,111,0.000000,0.000000,1,22,0,False
2,1467810917,2009-04-06 22:19:53-07:00,mattycus,@Kenichan I dived many times for the ball. Man...,89,0.500000,0.500000,1,22,0,False
3,1467811184,2009-04-06 22:19:57-07:00,ElleCTF,my whole body feels itchy and like its on fire,47,0.200000,0.400000,1,22,0,False
4,1467811193,2009-04-06 22:19:57-07:00,Karoli,"@nationwideclass no, it's not behaving at all....",111,-0.625000,1.000000,1,22,0,False


## Dependent variables

TODO: number of likes, comments, retweets from `post_id`

TODO: number of followers and friends from `author_id`

## Prediction

In [91]:
from sklearn.ensemble import RandomForestRegressor

In [93]:
X = tweets_dev[[
    'content_len',
    'content_polarity',
    'content_subjectivity',
    'day_of_week',
    'hour_of_day',
    'n_swears',
    'swear_presence',
    # TODO other features
]].values

In [ ]:
y = tweets_dev['n_likes']  # TODO